In [0]:
# Imports section
import torch, torchvision, PIL, json
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torch import nn, optim

In [0]:
# uninstall older PIL version
!pip uninstall -y Pillow
# install the new one
!pip install Pillow
# check Pillow version
# restart might be needed to use the new version
print(PIL.PILLOW_VERSION)

In [0]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

# If CUDA is not available, it can be enabled from 
# Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

In [0]:
# Download the Flower Dataset from https://www.kaggle.com/lenine/flower-102diffspecies-dataset
# Also, the category to name json file
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
# Set the training and validation directories
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# Define the transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.RandomRotation(20),
                                       transforms.Resize(225),
                                       transforms.CenterCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ColorJitter(hue=.05, saturation=.05),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

In [0]:
# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

In [0]:
# Define the dataloaders using the image datasets and the trainforms
batch_size = 32
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)

In [0]:
# Define a method to check the images from the dataloaders
def imshow(img):
  plt.figure(figsize=(25,15))
  img=img.numpy()
  img[0,:,:]=img[0,:,:]*0.229+0.485
  img[1,:,:]=img[1,:,:]*0.224+0.456
  img[2,:,:]=img[2,:,:]*0.225+0.406
  plt.imshow(np.transpose(img,(1,2,0)))

In [0]:
# Check the trainloader data
# TODO: Reposition the grid so that it splits the images
images,labels=next(iter(trainloader))
img=torchvision.utils.make_grid(images[:32])
imshow(img)

In [0]:
# Check the validloader data
# TODO: Reposition the grid so that it splits the images
images,labels=next(iter(validloader))
img=torchvision.utils.make_grid(images[:32])
imshow(img)

In [0]:
# Import category to name JSON data
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# Load a pretrained model of your choice from torchvision.models
model = models.resnet152(pretrained=True)

# Check model architecture so you can define your new classifier
print(model)

In [0]:
# Define a new untrained feed-forward classifier, using ReLu activations and droptout
# Keep input features at 2048, change output features from 1000 to 102 (# of flower categories)

model.fc = nn.Sequential(nn.Linear(2048, 512),
                            nn.ReLU(),
                            nn.Dropout(0.2),
                            nn.Linear(512, 102),
                            nn.LogSoftmax(dim=1))

# Check model with the new fc classifier
print(model)

In [0]:
# Freeze all model parameters except the untrained newly created classifier
for param in model.parameters():
    param.requires_grad = False
   
for param in model.fc.parameters():
    param.requires_grad = True

In [0]:
# Define the criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [0]:
# Set the device on which the training will be performed: cuda or cpu
# If cuda is available it's best to use it to improve trainig speed by over 100x
device=torch.device(("cuda") if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [0]:
# Train the model's classifier
epochs = 100
steps = 0
training_loss = 0
print_every = 20
valid_loss_min = np.Inf
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the device
        inputs, labels = inputs.to(device), labels.to(device)    
        optimizer.zero_grad()
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        # Validate model and print results after print_every number of steps
        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model(inputs)
                    batch_loss = criterion(logps, labels)
                    valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            valid_loss = valid_loss/len(validloader)
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {training_loss/print_every:.3f}.. "
                  f"Test loss: {valid_loss:.3f}.. "
                  f"Test accuracy: {accuracy/len(validloader):.3f}")
            # Save the model if validation loss has decreased
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                model.class_to_idx = train_data.class_to_idx
                torch.save({
                  'arch':'resnet152',
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'validation_accuraccy': {accuracy/len(validloader)},
                  'class_to_idx': model.class_to_idx,
                  'validation_loss': {valid_loss},
                  'train_loss': {training_loss/print_every},
                  }, 'checkpoint.pth')
                valid_loss_min = valid_loss
            training_loss = 0
            model.train()